In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [3]:
scSpark = SparkSession \
    .builder \
    .appName("Simple ETL") \
    .config("spark.jars", "postgresql-42.2.23.jar") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/xyz_comp.people") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/thinkpad/sekolah/sekolah/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/thinkpad/.ivy2/cache
The jars for the packages stored in: /home/thinkpad/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e7c24edb-dbba-4ddc-a381-4e9d689a7149;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
downloading https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/3.0.1/mongo-spark-connector_2.12-3.0.1.jar ...
	[SUCCESSFUL ] org.mongodb.spark#mongo-spark-connector_2.12;3.0.1!mongo-spark-connector_2.12.jar (906ms)
downloading https://repo1.maven.org/maven2/org/mongodb/mongodb-driver-sync/4.0.5/mongodb-driver-sync-4.0.5.jar ...
	[SUCCESSFUL ] org.mongodb#mongodb-driver-sync;4.0.5!mongodb-driver-sync.jar (224ms)
downloading https://repo1.mave

In [4]:
df = scSpark.read \
    .format("jdbc") \
    .option('url', 'jdbc:postgresql://localhost/xyz_comp') \
    .option('dbtable', 'people') \
    .option('user', 'postgres') \
    .option('password', 'admin') \
    .option('driver', 'org.postgresql.Driver') \
    .load()

In [5]:
df_job = df.groupBy("job").count() 
df_final = df_job.select(col("job"), col("count").alias("jumlah"))

In [6]:
df_final.write \
    .format("com.mongodb.spark.sql.DefaultSource") \
    .option('mode', 'append') \
    .save()

21/08/09 21:42:30 INFO cluster: Cluster created with settings {hosts=[127.0.0.1:27017], mode=SINGLE, requiredClusterType=UNKNOWN, serverSelectionTimeout='30000 ms'}
21/08/09 21:42:30 INFO MongoClientCache: Creating MongoClient: [127.0.0.1:27017]
21/08/09 21:42:30 INFO cluster: Cluster description not yet available. Waiting for 30000 ms before timing out
21/08/09 21:42:30 INFO connection: Opened connection [connectionId{localValue:1, serverValue:3}] to 127.0.0.1:27017
21/08/09 21:42:30 INFO cluster: Monitor thread successfully connected to server with description ServerDescription{address=127.0.0.1:27017, type=STANDALONE, state=CONNECTED, ok=true, minWireVersion=0, maxWireVersion=9, maxDocumentSize=16777216, logicalSessionTimeoutMinutes=30, roundTripTimeNanos=26032146}
21/08/09 21:42:30 INFO connection: Opened connection [connectionId{localValue:2, serverValue:4}] to 127.0.0.1:27017
21/08/09 21:42:34 INFO CodeGenerator: Code generated in 1315.506934 ms
21/08/09 21:42:34 INFO CodeGenerat